In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pyomo import environ as pe

In [2]:
# Create new model
tmp_model = pe.ConcreteModel()

In [3]:
# Initialize some sets
workers = (
    "Adam",
    "Bob",
    "Cynthia",
)

customers = (
    "Frank",
    "Ginny",
    "Hannah",
)

days = (
    'Sunday',
    'Monday',
    'Tuesday',
    'Wednesday',
    'Thursday',
    'Friday',
    'Saturday'
)

In [4]:
# Initialize schedules
worker_schedules = dict(
    ((w, d), 0)
    for w in workers
    for d in days
)
worker_schedules[('Adam', 'Sunday')] = 1
worker_schedules[('Adam', 'Monday')] = 1
worker_schedules[('Bob', 'Monday')] = 1
worker_schedules[('Bob', 'Tuesday')] = 1
worker_schedules[('Cynthia', 'Tuesday')] = 1
worker_schedules[('Cynthia', 'Wednesday')] = 1
{k: v for k, v in worker_schedules.items() if v == 1}

{('Adam', 'Sunday'): 1,
 ('Adam', 'Monday'): 1,
 ('Bob', 'Monday'): 1,
 ('Bob', 'Tuesday'): 1,
 ('Cynthia', 'Tuesday'): 1,
 ('Cynthia', 'Wednesday'): 1}

In [5]:
# Initialize schedules
customer_requests = dict(
    ((c, d), 0)
    for c in customers
    for d in days
)
customer_requests[('Frank', 'Monday')] = 1
customer_requests[('Ginny', 'Tuesday')] = 1
customer_requests[('Hannah', 'Wednesday')] = 1
{k: v for k, v in customer_requests.items() if v == 1}

{('Frank', 'Monday'): 1, ('Ginny', 'Tuesday'): 1, ('Hannah', 'Wednesday'): 1}

In [6]:
# Define our sets
tmp_model.workers = pe.Set(initialize=workers)
tmp_model.customers = pe.Set(initialize=customers)
tmp_model.days = pe.Set(initialize=days)

In [7]:
# Define our parameters
tmp_model.worker_schedules = pe.Param(
    tmp_model.workers * tmp_model.days,
    initialize = worker_schedules,
    within = pe.Binary
)

tmp_model.customer_requests = pe.Param(
    # On pyomo Set objects, the * operator returns the Cartesian product
    tmp_model.customers * tmp_model.days,
    # The dictionary mapping (channel, viewer) pairs to likelihood of viewership
    initialize = customer_requests,
    # Happiness probabilities are real numbers between 0 and 1
    within = pe.Binary
)

In [8]:
# Define our variables
tmp_model.assignments = pe.Var(
    # Defined over the channel matrix
    tmp_model.workers * tmp_model.customers * tmp_model.days,
    # Possible values are 0 and 1
    domain = pe.Binary,
    # Initialize to 0
    initialize = 0
)

In [9]:
# Define our constraints
# -> Here, 
#    1) each customer assigned to only 1 worker

def one_worker_per_customer(model, customer):
    n_workers_assigned_to_customer = sum(
        model.assignments[worker, customer, day]
        for worker in model.workers
        for day in model.days
    )
    return n_workers_assigned_to_customer == 1

tmp_model.one_worker_per_customer = pe.Constraint(
    tmp_model.customers,
    rule = one_worker_per_customer
)

In [10]:
# Define our objective
# -> Here the total number of matches

def get_legit_matches(model):
    # Multiply assignments by the 
    legit_matches = [
        model.assignments[w, c, d] * model.worker_schedules[w, d] * model.customer_requests[c, d]
        for (w, c, d), p in model.assignments.items()
    ]
    
    return sum(legit_matches)

tmp_model.objective = pe.Objective(
    rule = get_legit_matches,
    sense = pe.maximize
)

In [11]:
# Solve it!

#import sys
#sys.path.append('/mnt/c/glpk-4.65/w64/')

# Swap out "glpk" for "cbc" or "gurobi" if using another solver
solver = pe.SolverFactory("glpk")
# Add the keyword arg tee=True for a detailed trace of the solver's work.
tmp_solution = solver.solve(
    tmp_model,
    #executable = '/mnt/c/glpk-4.65/w64/glpsol'
)

In [12]:
tmp_solution

{'Problem': [{'Name': 'unknown', 'Lower bound': 3.0, 'Upper bound': 3.0, 'Number of objectives': 1, 'Number of constraints': 4, 'Number of variables': 64, 'Number of nonzeros': 64, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.0069582462310791016}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [13]:
# Inspect channel buys
for combo, matched in tmp_model.assignments.get_values().items():
    if matched == 1:
        print(combo, matched)

('Adam', 'Frank', 'Monday') 1.0
('Bob', 'Ginny', 'Tuesday') 1.0
('Cynthia', 'Hannah', 'Wednesday') 1.0


# Repeat but using AbstractModel instead for easier scenario testing

In [14]:
import scheduling_abstract_model as sam

In [15]:
abstract = sam.create()

In [16]:
# Scenario 1: Each customer only overlaps with one tutor
scenario1 = abstract.create_instance(data={
    None: {
        'worker_schedules': {
            ('Adam', 'Wednesday'): 1,
            ('Bob', 'Monday'): 1,
            ('Cynthia', 'Tuesday'): 1,
        },
        'customer_requests': {
            ('Frank', 'Monday'): 1,
            ('Ginny', 'Tuesday'): 1,
            ('Hannah', 'Wednesday'): 1,
        }
    }
})

scenario1_expected_output = {
    ('Bob', 'Frank', 'Monday'),
    ('Cynthia', 'Ginny', 'Tuesday'),
    ('Adam', 'Hannah', 'Wednesday')
}

solver.solve(scenario1)

assert sam.get_matches(scenario1) == scenario1_expected_output

In [17]:
# Scenario 2: More overlap between customers and workers, but still only one solution because
#             Hannah only overlaps with one worker
scenario2 = abstract.create_instance(data={
    None: {
        'worker_schedules': {
            ('Adam', 'Sunday'): 1,
            ('Adam', 'Monday'): 1,
            ('Bob', 'Monday'): 1,
            ('Bob', 'Tuesday'): 1,
            ('Cynthia', 'Tuesday'): 1,
            ('Cynthia', 'Wednesday'): 1,
        },
        'customer_requests': {
            ('Frank', 'Monday'): 1,
            ('Ginny', 'Tuesday'): 1,
            ('Hannah', 'Wednesday'): 1,
        }
    }
})

scenario2_expected_output = {
    ('Adam', 'Frank', 'Monday'),
    ('Bob', 'Ginny', 'Tuesday'),
    ('Cynthia', 'Hannah', 'Wednesday')
}

solver.solve(scenario2)

assert sam.get_matches(scenario2) == scenario2_expected_output

In [18]:
# Scenario 3: One customer won't get assigned
scenario3 = abstract.create_instance(data={
    None: {
        'worker_schedules': {
            ('Adam', 'Monday'): 1,
            ('Bob', 'Tuesday'): 1,
        },
        'customer_requests': {
            ('Frank', 'Monday'): 1,
            ('Ginny', 'Tuesday'): 1,
        }
    }
})

scenario3_expected_output = {
    ('Adam', 'Frank', 'Monday'),
    ('Bob', 'Ginny', 'Tuesday'),
}

solver.solve(scenario3)

assert sam.get_matches(scenario3) == scenario3_expected_output

In [19]:
sam.get_matches(scenario3)

{('Adam', 'Frank', 'Monday'), ('Bob', 'Ginny', 'Tuesday')}